In [1]:
# ruff: noqa: F401, E402, T201, T203, D103

from __future__ import annotations

import math
import sys
import time
from bisect import bisect_left, bisect_right
from collections import Counter, deque
from functools import cmp_to_key, lru_cache, reduce
from heapq import heapify, heappop, heappush, heappushpop, nsmallest
from importlib import reload
from itertools import (
    accumulate,
    chain,
    combinations,
    islice,
    pairwise,
    permutations,
    product,
    repeat,
    starmap,
    tee,
)
from math import comb, exp, factorial, inf, log, prod, sqrt
from operator import add, and_, contains, indexOf, itemgetter, mul, neg, or_, xor
from pprint import pformat, pprint
from typing import Callable, Iterable, Iterator, List, Tuple

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from IPython.display import HTML, clear_output, display

import arrays
import bits
import combinatorics
import graphs
import grid
import lists
import mathematics
import matrix
import parsing
import search
import sequences
import sets
import sorting
import stack
import strings
import trees

for m in (
    arrays,
    bits,
    combinatorics,
    graphs,
    grid,
    lists,
    mathematics,
    matrix,
    parsing,
    search,
    sequences,
    sets,
    sorting,
    stack,
    strings,
    trees,
):
    reload(m)


from combinatorics import fib
from mathematics import prime_numbers as primes
from search import lower_index, upper_index
from sequences import find_if
from strings import splint

np.set_printoptions(edgeitems=5, precision=3, linewidth=300)

In [205]:
def shortest_path_with_special_edge(edges: list, a: int, b: int) -> int:
    n = max(max(e[0], e[1]) for e in edges) + 1
    adj = [[] for _ in range(n)]
    for e in edges:
        adj[e[0]].append(e[1:])
        adj[e[1]].append((e[0], e[2], e[3]))
    mx = sum(e[2] for e in edges) + 1
    # Minimum visited distance.
    v = [[mx, mx] for _ in range(n)]
    # Priority heap queue.
    q = [(0, 0, a)]
    while q:
        # Distance, number of special edges, desitination node.
        d, s, n = heappop(q)
        # Cut if shorter value found.
        if v[n][s] <= d:
            continue
        # This will be the min distance to b.
        if n == b:
            return d
        # Update minimum visited distance.
        v[n][s] = v[n][1] = d
        for c, e1, e2 in adj[n]:
            # Cut, if there exists a better path.
            if v[c][s] > d + e1:
                heappush(q, (d + e1, s, c))
                # In case we did not use the curved edge yet,
                # and it is shorter than the stright one.
                if s == 0 and e2 < e1 and v[c][1] > d + e2:
                    heappush(q, (d + e2, 1, c))
    return -1

In [209]:
shortest_path_with_special_edge([[1, 2, 4, 1]], 1, 2)

1

In [210]:
edges = [
    [158, 79, 24, 72],
    [172, 103, 6, 35],
    [22, 93, 17, 90],
    [65, 155, 37, 25],
    [153, 45, 5, 50],
    [73, 70, 80, 21],
    [124, 23, 6, 75],
    [169, 32, 93, 71],
    [106, 38, 87, 61],
    [104, 98, 81, 83],
    [143, 11, 41, 65],
    [84, 120, 31, 33],
    [77, 71, 89, 11],
    [143, 25, 21, 69],
    [125, 149, 83, 46],
    [46, 55, 11, 44],
    [67, 94, 92, 99],
    [158, 16, 74, 44],
    [143, 34, 6, 30],
    [65, 165, 15, 52],
    [74, 100, 11, 84],
    [20, 42, 54, 26],
    [133, 97, 77, 23],
    [89, 24, 79, 59],
    [67, 153, 72, 13],
    [113, 124, 88, 79],
    [82, 144, 35, 14],
    [142, 63, 20, 38],
    [130, 105, 71, 85],
    [70, 136, 38, 72],
    [115, 177, 99, 75],
    [35, 91, 8, 74],
    [98, 18, 67, 79],
    [61, 160, 40, 15],
    [55, 45, 88, 23],
    [33, 73, 7, 80],
    [44, 163, 46, 14],
    [39, 51, 57, 80],
    [46, 85, 62, 49],
    [65, 25, 99, 72],
    [13, 55, 88, 50],
    [111, 79, 81, 4],
    [97, 46, 35, 43],
    [53, 144, 88, 25],
    [49, 91, 19, 80],
    [119, 132, 20, 49],
    [117, 104, 22, 33],
    [141, 105, 15, 34],
    [136, 132, 56, 57],
    [156, 56, 77, 63],
    [151, 114, 85, 70],
    [95, 85, 2, 81],
    [147, 24, 84, 66],
    [80, 76, 59, 75],
    [150, 80, 10, 31],
    [68, 119, 37, 63],
    [11, 12, 67, 77],
    [24, 21, 24, 99],
    [100, 22, 89, 78],
    [145, 18, 46, 20],
    [172, 129, 66, 9],
    [136, 175, 29, 83],
    [137, 48, 62, 56],
    [163, 76, 8, 63],
    [164, 21, 52, 28],
    [137, 145, 7, 16],
    [63, 88, 36, 21],
    [120, 59, 71, 7],
    [171, 9, 75, 80],
    [36, 125, 79, 55],
    [102, 30, 23, 67],
    [124, 114, 25, 62],
    [117, 98, 55, 44],
    [125, 109, 34, 82],
    [147, 13, 93, 63],
    [7, 148, 90, 69],
    [3, 135, 55, 3],
    [173, 16, 77, 74],
    [36, 9, 45, 1],
    [69, 38, 80, 53],
    [43, 11, 93, 68],
    [81, 80, 4, 58],
    [74, 87, 30, 10],
    [55, 23, 34, 95],
    [71, 60, 26, 78],
    [10, 2, 18, 13],
    [165, 3, 19, 4],
    [51, 93, 16, 87],
    [42, 3, 22, 89],
    [34, 68, 74, 63],
    [152, 12, 27, 52],
    [121, 36, 64, 68],
    [33, 54, 47, 14],
    [72, 170, 22, 42],
    [127, 127, 14, 70],
    [36, 102, 38, 13],
    [135, 172, 5, 6],
    [55, 169, 23, 50],
    [148, 5, 5, 88],
    [102, 167, 95, 100],
    [162, 174, 86, 66],
    [48, 93, 30, 92],
    [149, 171, 2, 71],
    [76, 109, 41, 38],
    [21, 25, 47, 63],
    [176, 12, 92, 30],
    [149, 116, 24, 59],
    [54, 13, 100, 95],
    [119, 132, 20, 2],
    [110, 11, 42, 88],
    [93, 46, 62, 22],
    [138, 18, 42, 66],
    [122, 167, 27, 85],
    [161, 44, 73, 75],
    [169, 161, 51, 88],
    [53, 165, 11, 90],
    [1, 114, 29, 35],
    [123, 1, 49, 84],
    [10, 49, 18, 15],
    [147, 23, 16, 61],
    [141, 55, 98, 70],
    [172, 21, 53, 53],
    [120, 142, 95, 7],
    [133, 42, 11, 12],
    [45, 162, 36, 88],
    [107, 76, 32, 31],
    [117, 87, 31, 22],
    [68, 50, 32, 52],
    [144, 44, 51, 33],
    [44, 12, 8, 29],
    [146, 25, 71, 60],
    [86, 54, 49, 53],
    [127, 56, 6, 93],
    [82, 111, 94, 34],
    [90, 8, 5, 95],
    [5, 65, 78, 58],
    [59, 167, 23, 95],
    [117, 77, 39, 58],
    [29, 162, 72, 4],
    [101, 124, 92, 98],
    [177, 11, 74, 99],
    [138, 71, 68, 3],
    [14, 46, 15, 95],
    [102, 124, 69, 63],
    [130, 174, 86, 26],
    [5, 58, 2, 92],
    [140, 117, 58, 6],
    [171, 38, 1, 94],
    [95, 116, 21, 61],
    [97, 4, 70, 69],
    [143, 71, 14, 30],
    [130, 143, 41, 11],
    [86, 51, 55, 41],
    [116, 36, 80, 48],
    [59, 18, 68, 11],
    [133, 69, 23, 81],
    [144, 4, 62, 51],
    [48, 25, 11, 92],
    [143, 90, 57, 46],
    [83, 151, 60, 75],
    [19, 98, 87, 28],
    [65, 159, 46, 97],
    [136, 131, 46, 9],
    [156, 153, 67, 32],
    [68, 114, 26, 82],
    [14, 89, 60, 29],
    [135, 76, 45, 77],
    [41, 63, 7, 87],
    [34, 28, 77, 92],
    [109, 81, 60, 79],
    [175, 157, 13, 2],
    [124, 7, 65, 73],
    [157, 32, 45, 76],
    [87, 153, 72, 31],
    [33, 14, 49, 62],
    [66, 35, 29, 77],
    [48, 140, 62, 48],
    [172, 124, 45, 23],
    [19, 163, 93, 70],
    [10, 51, 72, 3],
    [133, 147, 94, 76],
    [137, 146, 31, 10],
    [45, 50, 82, 54],
    [86, 109, 92, 35],
    [12, 120, 53, 52],
    [25, 93, 9, 24],
    [27, 45, 21, 35],
    [50, 149, 33, 3],
    [131, 42, 17, 91],
    [78, 129, 79, 7],
    [129, 140, 68, 16],
    [118, 86, 20, 44],
    [46, 40, 35, 13],
    [171, 59, 21, 34],
    [174, 129, 38, 71],
    [88, 166, 51, 27],
    [121, 79, 8, 51],
    [159, 29, 74, 20],
    [12, 6, 30, 88],
    [114, 112, 30, 53],
    [79, 93, 68, 42],
    [160, 5, 41, 38],
    [136, 108, 92, 22],
    [8, 73, 87, 50],
    [170, 101, 92, 59],
    [142, 36, 40, 52],
    [7, 133, 73, 58],
    [115, 161, 58, 68],
    [54, 124, 83, 67],
    [8, 151, 16, 79],
    [87, 126, 70, 14],
    [6, 82, 60, 77],
    [95, 137, 8, 63],
    [75, 50, 54, 88],
    [171, 16, 4, 59],
    [98, 1, 3, 92],
    [100, 145, 52, 29],
    [131, 100, 55, 61],
    [82, 8, 68, 85],
    [147, 163, 43, 74],
    [11, 3, 30, 28],
    [75, 35, 52, 18],
    [111, 67, 40, 98],
    [103, 63, 17, 22],
    [30, 153, 93, 7],
    [15, 177, 53, 52],
    [142, 48, 91, 82],
    [120, 103, 35, 71],
    [65, 10, 78, 21],
    [62, 41, 25, 21],
    [131, 26, 4, 63],
    [108, 114, 46, 52],
    [153, 89, 23, 65],
    [144, 135, 99, 81],
    [100, 19, 83, 34],
    [35, 176, 93, 56],
    [42, 147, 9, 31],
    [154, 31, 89, 42],
    [170, 32, 29, 87],
    [3, 4, 24, 65],
    [83, 77, 41, 75],
    [6, 143, 37, 3],
    [24, 14, 27, 44],
    [68, 166, 65, 80],
    [93, 147, 23, 48],
    [93, 5, 54, 47],
    [25, 106, 19, 81],
    [118, 27, 43, 13],
    [128, 78, 13, 54],
    [40, 37, 38, 71],
    [125, 112, 16, 28],
    [176, 143, 54, 23],
    [47, 149, 59, 14],
    [22, 30, 81, 6],
    [144, 78, 17, 65],
    [14, 163, 52, 5],
    [127, 167, 83, 65],
    [135, 136, 26, 96],
    [96, 48, 81, 14],
    [51, 21, 21, 57],
    [57, 36, 30, 49],
    [38, 104, 29, 43],
    [172, 72, 11, 40],
    [176, 101, 58, 44],
    [58, 77, 36, 81],
    [69, 118, 18, 86],
    [170, 51, 63, 75],
    [136, 20, 85, 86],
    [84, 76, 23, 79],
    [108, 115, 40, 81],
    [94, 71, 62, 85],
    [35, 174, 26, 30],
    [92, 61, 96, 89],
    [17, 32, 58, 77],
    [64, 162, 10, 62],
    [170, 3, 91, 39],
    [151, 127, 67, 14],
    [23, 142, 50, 68],
    [18, 172, 70, 56],
    [78, 67, 12, 60],
    [54, 132, 94, 17],
    [4, 60, 41, 98],
    [169, 9, 41, 36],
    [89, 145, 3, 76],
    [90, 53, 29, 5],
    [103, 69, 88, 16],
    [110, 131, 42, 54],
    [65, 40, 91, 85],
    [139, 9, 35, 73],
    [30, 70, 24, 44],
    [92, 158, 52, 3],
    [119, 174, 11, 15],
    [53, 63, 54, 85],
    [117, 174, 38, 66],
    [37, 21, 27, 90],
    [48, 108, 36, 93],
    [128, 10, 34, 78],
    [96, 43, 87, 31],
    [26, 56, 44, 17],
    [87, 167, 98, 70],
    [99, 149, 58, 93],
    [57, 22, 7, 49],
    [144, 79, 49, 24],
    [73, 150, 70, 92],
    [107, 4, 76, 63],
    [72, 128, 65, 44],
    [64, 47, 29, 74],
    [21, 159, 70, 52],
    [27, 138, 64, 17],
    [70, 111, 28, 61],
    [31, 43, 35, 88],
    [60, 155, 46, 59],
    [36, 84, 88, 36],
    [163, 102, 18, 89],
    [161, 35, 54, 13],
    [126, 71, 88, 98],
    [72, 177, 37, 21],
    [170, 133, 88, 26],
    [87, 56, 25, 67],
    [57, 129, 72, 93],
    [2, 117, 7, 86],
    [38, 113, 67, 41],
    [12, 7, 80, 21],
    [100, 42, 39, 39],
    [58, 173, 48, 50],
    [129, 7, 4, 65],
    [139, 32, 28, 16],
    [162, 13, 79, 59],
    [166, 146, 40, 76],
    [78, 41, 52, 29],
    [79, 41, 35, 97],
    [103, 143, 100, 3],
    [44, 139, 10, 23],
    [9, 131, 11, 22],
    [46, 85, 64, 72],
    [117, 11, 87, 80],
    [39, 142, 5, 18],
    [2, 56, 40, 83],
    [128, 27, 94, 14],
    [80, 158, 6, 46],
    [55, 110, 10, 9],
    [66, 174, 70, 71],
    [142, 56, 24, 50],
    [150, 168, 81, 8],
    [52, 39, 23, 33],
    [15, 119, 20, 22],
    [91, 44, 33, 94],
    [39, 85, 14, 49],
    [152, 26, 73, 15],
    [55, 177, 1, 50],
    [111, 83, 9, 10],
    [73, 107, 61, 91],
    [48, 43, 60, 56],
    [11, 42, 39, 67],
    [89, 132, 41, 68],
    [55, 35, 8, 59],
    [5, 120, 2, 65],
    [104, 113, 79, 22],
    [3, 87, 59, 88],
    [94, 72, 54, 6],
    [106, 39, 52, 95],
    [22, 116, 2, 93],
    [55, 141, 78, 49],
    [161, 5, 59, 81],
    [53, 84, 39, 59],
    [123, 175, 37, 80],
    [11, 48, 52, 1],
    [49, 144, 81, 88],
    [68, 98, 78, 74],
    [166, 103, 4, 40],
    [111, 110, 79, 88],
    [124, 55, 99, 49],
    [16, 147, 29, 10],
    [39, 112, 98, 78],
    [32, 40, 47, 76],
    [145, 163, 50, 1],
    [61, 34, 59, 62],
]

In [211]:
shortest_path_with_special_edge(edges, 66, 30)

109